In [306]:
import os
import numpy as np
import pandas as pd

In [373]:
data = pd.read_csv('spam.csv', encoding="ISO-8859-1")
data = data[['v1', 'v2']]
data.columns = ['Label', 'Msg']

data.Label = data.Label.map({'ham': 0, 'spam': 1})
print(data.head(20))

    Label                                                Msg
0       0  Go until jurong point, crazy.. Available only ...
1       0                      Ok lar... Joking wif u oni...
2       1  Free entry in 2 a wkly comp to win FA Cup fina...
3       0  U dun say so early hor... U c already then say...
4       0  Nah I don't think he goes to usf, he lives aro...
5       1  FreeMsg Hey there darling it's been 3 week's n...
6       0  Even my brother is not like to speak with me. ...
7       0  As per your request 'Melle Melle (Oru Minnamin...
8       1  WINNER!! As a valued network customer you have...
9       1  Had your mobile 11 months or more? U R entitle...
10      0  I'm gonna be home soon and i don't want to tal...
11      1  SIX chances to win CASH! From 100 to 20,000 po...
12      1  URGENT! You have won a 1 week FREE membership ...
13      0  I've been searching for the right words to tha...
14      0                I HAVE A DATE ON SUNDAY WITH WILL!!
15      1  XXXMobileMovi

In [407]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

punctuations = string.punctuation
stopwords = set(stopwords.words('english'))

def preprocess_text(text):
    
    # Normalize text
    text = text.lower()
    
    # Strip non-ascii accents
    text = text.encode('ascii', 'ignore').decode('ascii') 
    
    # Retain specific words that could characterize spam msgs
    for word in ["t&c's", 't&cs', 't&c']:
        if word in text:
            text = text.replace(word, 'termsandconditions')
    
    def remove_punctuations(word):
        if (len(word)==2 or len(word)==3) and \
                all([ch in punctuations for ch in word]) and \
                not all([ch == word[0] for ch in word]):
            # Cond1 - Ensures smileys are only 2/3 char long
            # Cond2 - All chars of word are punctuations
            # Cond3 - All punctuations are not the same (Ex. || , ... , !! , etc)
            return "SMILEY"
        return ''.join([ch for ch in word if ch not in punctuations])
    
    # Tokenize & eliminate punctuations
    tokens = [remove_punctuations(word) for sent in sent_tokenize(text) for word in sent.split()]  # word_tokenize(sent)]
    
    # Remove numeric tokens
    tokens = [token for token in tokens if token and not token.isdigit()]  # len(token) >=2] # and (token.isalpha() or token.isdigit())]
    
    # Remove alphanumeric tokens
    # tokens = [token for token in tokens if not (token.isalnum() and not (token.isalpha() or token.isdigit()))]
    new_tokens = []
    for token in tokens:
        if not (token.isalnum() and not (token.isalpha() or token.isdigit())):
            new_tokens.append(token)
        else:
            new_tokens.append('alphanumeric')
    
    
    # Remove stopwords
    # tokens = [token for token in tokens if token not in stopwords]
    
    return ' '.join(new_tokens)
    
    

In [408]:
data['Processed_Msg'] = data['Msg'].apply(preprocess_text)
data['Msg_len'] = data['Msg'].apply(len)

In [378]:
# from sklearn.preprocessing import MinMaxScaler
# data['Msg_len'] = MinMaxScaler().fit_transform(data['Msg_len'].values.reshape(-1, 1))
# data

In [379]:
# data.Processed_Msg.head(20).values

In [380]:
data[data.Processed_Msg.apply(len) == 0]

,Label,Msg,Processed_Msg,Msg_len
1611,0,645,,3


In [381]:
## Dictribution of class labels
data['Label'].value_counts()

0    4825
1     747
Name: Label, dtype: int64

##### Import necessary sklearn libraries

In [409]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [410]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, adjusted_mutual_info_score
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, confusion_matrix

def print_scores(y_test, y_pred):
#     print(f'Precision : {precision_score(y_test, y_pred)}')
#     print(f'Recall : {recall_score(y_test, y_pred)}')
#     print(f'f1_score : {f1_score(y_test, y_pred)}')
    print(f'Accuracy : {accuracy_score(y_test, y_pred)}')
#     print(f'adjusted_mutual_info_score : {adjusted_mutual_info_score(y_test, y_pred)}')
#     print(f'roc_auc_score : {roc_auc_score(y_test, y_pred)}')
    print(f'\nclassification_report :\n {classification_report(y_test, y_pred)}')
    print(f'\nconfusion_matrix :\n {confusion_matrix(y_test, y_pred)}')

In [411]:
X = data.Processed_Msg
# X = data.Msg
y = data.Label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [393]:
# X = data[['Processed_Msg', 'Msg_len']]
# y = data.Label
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#### Using Tfidf Vectors

In [412]:
tfv = TfidfVectorizer()
train_vectors = tfv.fit_transform(X_train)
test_vectors = tfv.transform(X_test)

In [395]:
# tfv = TfidfVectorizer()
# train_vectors = tfv.fit_transform(X_train['Processed_Msg'])
# test_vectors = tfv.transform(X_test['Processed_Msg'])

# # Feature Stacking for train_vectors
# msg_len = X_train['Msg_len'].to_numpy()
# msg_len = msg_len.reshape((msg_len.shape[0],1))

# train_vectors = np.hstack((train_vectors.toarray(), msg_len))

# # Feature Stacking for test_vectors
# msg_len = X_test['Msg_len'].to_numpy()
# msg_len = msg_len.reshape((msg_len.shape[0],1))

# test_vectors = np.hstack((test_vectors.toarray(), msg_len))

In [413]:
# Support Vector classifier 
svc = SVC()
svc.fit(train_vectors, y_train)
y_pred = svc.predict(test_vectors)
print_scores(y_test, y_pred)

Accuracy : 0.9820627802690582

classification_report :
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.99      0.88      0.93       150

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115


confusion_matrix :
 [[963   2]
 [ 18 132]]


In [420]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(train_vectors, y_train)
y_pred = rfc.predict(test_vectors)
print_scores(y_test, y_pred)

Accuracy : 0.9687724335965542

classification_report :
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      2413
           1       0.95      0.81      0.87       373

    accuracy                           0.97      2786
   macro avg       0.96      0.90      0.93      2786
weighted avg       0.97      0.97      0.97      2786


confusion_matrix :
 [[2397   16]
 [  71  302]]


#### Using Count Vectors

In [414]:
cv = CountVectorizer(decode_error='ignore', stop_words='english', ngram_range=(1,2))
train_vectors = cv.fit_transform(X_train)
test_vectors = cv.transform(X_test)

In [398]:
# cv = CountVectorizer(decode_error='ignore', stop_words='english', ngram_range=(1,2))
# train_vectors = cv.fit_transform(X_train['Processed_Msg'])
# test_vectors = cv.transform(X_test['Processed_Msg'])

# # Feature Stacking for train_vectors
# msg_len = X_train['Msg_len'].to_numpy()
# msg_len = msg_len.reshape((msg_len.shape[0],1))

# train_vectors = np.hstack((train_vectors.toarray(), msg_len))

# # Feature Stacking for test_vectors
# msg_len = X_test['Msg_len'].to_numpy()
# msg_len = msg_len.reshape((msg_len.shape[0],1))

# test_vectors = np.hstack((test_vectors.toarray(), msg_len))

In [399]:
#cv.get_feature_names()[:20], cv.get_feature_names()[-20:]

In [415]:
mnb_clf = MultinomialNB()
mnb_clf.fit(train_vectors, y_train)
y_pred = mnb_clf.predict(test_vectors)
print_scores(y_test, y_pred)

Accuracy : 0.9901345291479821

classification_report :
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.97      0.96      0.96       150

    accuracy                           0.99      1115
   macro avg       0.98      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115


confusion_matrix :
 [[960   5]
 [  6 144]]


In [416]:
from sklearn.model_selection import StratifiedShuffleSplit
splits = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)

for train_index, test_index in splits.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    tfv = TfidfVectorizer()
    train_vectors = tfv.fit_transform(X_train)
    test_vectors = tfv.transform(X_test)
    
    svc = SVC()
    svc.fit(train_vectors, y_train)
    y_pred = svc.predict(test_vectors)
    print("#############################")
    print_scores(y_test, y_pred)
    

#############################
Accuracy : 0.9766690595836325

classification_report :
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2413
           1       0.98      0.84      0.91       373

    accuracy                           0.98      2786
   macro avg       0.98      0.92      0.95      2786
weighted avg       0.98      0.98      0.98      2786


confusion_matrix :
 [[2406    7]
 [  58  315]]
#############################
Accuracy : 0.9763101220387652

classification_report :
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2413
           1       0.98      0.84      0.90       373

    accuracy                           0.98      2786
   macro avg       0.98      0.92      0.95      2786
weighted avg       0.98      0.98      0.98      2786


confusion_matrix :
 [[2407    6]
 [  60  313]]
#############################
Accuracy : 0.9770279971284996

classification_re

In [417]:
splits = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)

for train_index, test_index in splits.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    cv = CountVectorizer(decode_error='ignore', stop_words='english', ngram_range=(1,2))
    train_vectors = cv.fit_transform(X_train)
    test_vectors = cv.transform(X_test)
    
    mnb_clf = MultinomialNB()
    mnb_clf.fit(train_vectors, y_train)
    y_pred = mnb_clf.predict(test_vectors)
    print("#############################")
    print_scores(y_test, y_pred)

#############################
Accuracy : 0.9802584350323044

classification_report :
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      2413
           1       0.95      0.90      0.92       373

    accuracy                           0.98      2786
   macro avg       0.97      0.95      0.96      2786
weighted avg       0.98      0.98      0.98      2786


confusion_matrix :
 [[2395   18]
 [  37  336]]
#############################
Accuracy : 0.9827709978463748

classification_report :
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      2413
           1       0.96      0.91      0.93       373

    accuracy                           0.98      2786
   macro avg       0.97      0.95      0.96      2786
weighted avg       0.98      0.98      0.98      2786


confusion_matrix :
 [[2399   14]
 [  34  339]]
#############################
Accuracy : 0.9806173725771715

classification_re